# Tutorial 14: Operations & Lineage Explorer

Learn how to:
- Retrieve recent operation history
- Filter operations by status or type
- Inspect computational lineage graphs for tensors

All requests fall back to demo payloads when the API is unavailable.

In [ ]:
from tutorial_utils import ping_server, pretty_json
import requests

API = "http://127.0.0.1:7860"
SERVER = ping_server(API)
print(f"\ud83d\udce1 Tensorus server available: {SERVER}")


## Recent operations
Fetch the latest operations via `/api/v1/operations/recent`.

In [ ]:
if SERVER:
    try:
        resp = requests.get(f"{API}/api/v1/operations/recent", timeout=10)
        resp.raise_for_status()
        recent_ops = resp.json()
    except requests.RequestException as exc:
        recent_ops = {"error": str(exc), "demo_mode": True}
else:
    recent_ops = {"demo_mode": True, "operations": [{"operation": "tensor_add", "status": "success", "tensor_ids": ["demo_a", "demo_b"]}]}

print(pretty_json(recent_ops))


## Filter by status
You can filter by `status` or `operation_type` using query parameters.

In [ ]:
params = {"status": "success"}

if SERVER:
    try:
        filtered = requests.get(f"{API}/api/v1/operations/recent", params=params, timeout=10)
        filtered.raise_for_status()
        filtered_ops = filtered.json()
    except requests.RequestException as exc:
        filtered_ops = {"error": str(exc), "demo_mode": True}
else:
    filtered_ops = {"demo_mode": True, "operations": [{"operation": "tensor_norm", "status": "success"}]}

print(pretty_json(filtered_ops))


## Tensor lineage
Use `/api/v1/lineage/tensor/{tensor_id}` to trace how a tensor was produced.

In [ ]:
tensor_id = "demo_tensor"

if SERVER:
    try:
        lineage_resp = requests.get(f"{API}/api/v1/lineage/tensor/{tensor_id}", timeout=10)
        lineage_resp.raise_for_status()
        lineage_data = lineage_resp.json()
    except requests.RequestException as exc:
        lineage_data = {"error": str(exc), "demo_mode": True}
else:
    lineage_data = {"demo_mode": True, "lineage": [{"operation": "tensor_add", "inputs": ["base_1", "base_2"], "output": tensor_id}]}

print(pretty_json(lineage_data))


## DOT graph download
Lineage graphs can be exported in DOT format for visualization tools like Graphviz.

In [ ]:
if SERVER:
    try:
        dot_resp = requests.get(f"{API}/api/v1/lineage/tensor/{tensor_id}/dot", timeout=10)
        dot_resp.raise_for_status()
        lineage_dot = dot_resp.text
    except requests.RequestException as exc:
        lineage_dot = f"Error retrieving DOT: {exc}"
else:
    lineage_dot = "digraph demo { base_1 -> demo_tensor; base_2 -> demo_tensor; }"

print(lineage_dot)
